### Recommend Recipes By Contents Based Filtering 


In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount('/gdrive')

recipes=pd.read_excel('/gdrive/My Drive/Colab Notebooks/DataHandling/recipes_recommend.xlsx')
print(type(recipes))
recipes.head(10)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>


,seq,section,title,parts
0,1,food,칼륨 듬뿍 고구마죽,고구마 설탕 찹쌀가루 물 잣
1,2,food,누룽지 두부 계란죽,애호박 표고버섯 당근 누룽지 순두부 달걀 참기름 소금 참깨 흰후추
2,3,food,오색지라시 스시,밥 식초 설탕 소금 달걀노른자 표고버섯 새송이버섯 새우 홍피망 청피망 양파 대두유 ...
3,4,food,두부 곤약 나물 비빔밥,두부 흰쌀 현미쌀 찹쌀 실곤약 콩나물 표고버섯 애호박 고사리 당근 소금 소금나물데침...
4,5,food,간장을 이용한 닭개장 비빔밥,쌀 검은쌀 닭가슴살 월계수잎 후추 숙주 토란대 고사리 달걀 대파 들기름 실고추 소금...
5,6,food,취나물 비빔밥 & 청국장 쌈장,밥 쇠고기 취나물 양파 마늘 간장 매실즙 참기름 참깨 소금 청국장 된장 콩가루 멸치...
6,7,food,해물 채소장을 넣은 곰취 쌈밥,잡곡밥 곰취 소금 조갯살 새우살 오징어살 감자 애호박 양파 청양고추 표고버섯 대파 ...
7,8,food,단호박 케일밥,찹쌀 밤 은행 잣 단호박 대추 케일 물 설탕
8,9,food,오색영양밥 롤,찹쌀 흑미 바지락 강낭콩 완두콩 메주콩 은행 달걀 애호박 가지 당근 표고버섯 대추 ...
9,10,food,새우 두부 계란찜,연두부 칵테일새우 달걀 생크림 설탕 버터 시금치


In [2]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
prefer_mat = count_vect.fit_transform(recipes['parts'])
print(prefer_mat.shape)

(1308, 10114)


In [3]:
from sklearn.metrics.pairwise import cosine_similarity

prefer_sim = cosine_similarity(prefer_mat, prefer_mat)
print(prefer_sim.shape)
print(prefer_sim)


(1308, 1308)
[[1.         0.         0.08944272 ... 0.         0.         0.        ]
 [0.         1.         0.13764944 ... 0.1338549  0.         0.19088543]
 [0.08944272 0.13764944 1.         ... 0.11669199 0.07698004 0.11094004]
 ...
 [0.         0.1338549  0.11669199 ... 1.         0.02807173 0.48546804]
 [0.         0.         0.07698004 ... 0.02807173 1.         0.05337605]
 [0.         0.19088543 0.11094004 ... 0.48546804 0.05337605 1.        ]]


In [4]:
# recipes_data = pd.DataFrame(prefer_sim)
# xlxs_dir = '/gdrive/My Drive/Colab Notebooks/DataHandling/recipesData.xlsx'

# with pd.ExcelWriter(xlxs_dir) as writer:
#      recipes_data.to_excel(writer, sheet_name = 'compare')

In [5]:
prefer_sim_sorted_ind = prefer_sim.argsort()[:, ::-1]
print(prefer_sim_sorted_ind)

[[   0 1200 1129 ...  693  692 1307]
 [   1  579  601 ...  433  434    0]
 [   2 1194  930 ... 1063 1244  477]
 ...
 [1305 1307  818 ...  937  938    0]
 [1306 1297 1281 ...  748  746    0]
 [1307 1305  818 ...  325  327    0]]


In [6]:
def find_sim_recipes(df, sorted_ind, my_refrigerator):
    
    # 인자로 입력된 plants DataFrame에서 'RCP_NM' 컬럼이 입력된 my_refrigerator 값인 DataFrame추출
    refrigerator = df[df['title'] == my_refrigerator]
    
    # my_refrigerator을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 prefer_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    refrigerator_index = refrigerator.index.values
    
    similar_indexes = sorted_ind[refrigerator_index]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    # print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]


In [7]:
similar_recipes = find_sim_recipes(recipes, prefer_sim_sorted_ind, '내 냉장고')
similar_recipes = similar_recipes[similar_recipes['section'] == 'food']
similar_recipes[:10]

,seq,section,title,parts
1305,1306,food,삼각김밥 빵,쌀가루 우유 물 르뱅발효종 달걀 소금 올리브유 양파 당근 먹물가루 토마토 양파 피망...
818,819,food,새우살토마토스튜,새우살 토마토 양파 당근 애호박 건다시마 버터 케찹 후추 소금 물
967,968,food,야채 오믈렛,양파 당근 홍피망 청피망 밀가루 베이킹파우더 설탕 달걀 우유 토마토케첩 식용유 소금
985,986,food,낫토 시래기 라이스전,밥 양파 당근 낫토 시래기 카레가루 파 밀가루 달걀 식용유 소금 후춧가루
1146,1147,food,펌킨크로켓,단호박 양파 당근 참나물 달걀 모짜렐라치즈 빵가루 식용유 소금 후춧가루
919,920,food,알파벳 스프,양파 샐러리 당근 마늘 식용유 닭육수 후추 소금 월계수잎 알파벳파스타
221,222,food,콩나물부추볶음,콩나물 부추 양파 당근 파 감자 소금 식용유
680,681,food,양송이크림볶음밥,닭고기살 양송이 떡볶이떡 양파 당근 소금 버터 밥 육수 우유 생크림 치즈
922,923,food,야채빵,밀가루 베이킹파우더 버터 달걀 설탕 당근 양파 캔옥수수 완두콩 우유 소금
1072,1073,food,호두 치킨,달각슴살 올리브오일 양파 당근 닭육수 호두 건포도 계피가루 큐민씨 소금 후추
